In [ ]:
# exploratory_analysis.ipynb

"""
This Jupyter notebook is used for exploratory analysis of the synthetic graph data and initial results of the algorithms.
It includes visualizations, statistical summaries, and initial experimentations to understand the dataset and ensure it meets desired conditions.
"""

# Import necessary libraries
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from ..src.graph_generation import generate_random_graphs
from ..src.ispf_models import run_dijkstra
from ..src.ml_models import build_gnn_model, train_gnn_model
import torch
import torch_geometric
from torch_geometric.data import DataLoader

# Generate Synthetic Graph Data
num_nodes = 50
edge_density = 0.2
weight_distribution = 'uniform'

# Create a random graph
graph = generate_random_graphs(num_nodes, edge_density, weight_distribution)

# Visualize the Generated Graph
plt.figure(figsize=(10, 8))
pos = nx.spring_layout(graph)
nx.draw(graph, pos, with_labels=True, node_color='skyblue', edge_color='gray', node_size=500, font_size=10)
nx.draw_networkx_edge_labels(graph, pos, edge_labels={(u, v): f'{d["weight"]:.1f}' for u, v, d in graph.edges(data=True)})
plt.title("Generated Random Graph")
plt.show()

# Graph Statistics
print("Graph Statistics:")
print(f"Number of nodes: {graph.number_of_nodes()}")
print(f"Number of edges: {graph.number_of_edges()}")
print(f"Average degree: {np.mean([deg for _, deg in graph.degree()])}")
print(f"Graph density: {nx.density(graph):.4f}")

# Degree Distribution
degrees = [deg for _, deg in graph.degree()]
plt.figure(figsize=(10, 6))
plt.hist(degrees, bins=range(1, max(degrees) + 2), edgecolor='black', align='left')
plt.title("Degree Distribution of the Generated Graph")
plt.xlabel("Degree")
plt.ylabel("Frequency")
plt.grid(axis='y', linestyle='--')
plt.show()

# Run Dijkstra's Algorithm
source_node = 0
shortest_paths = run_dijkstra(graph, source_node)

# Display Shortest Path Results
print(f"Shortest Paths from Node {source_node}:")
for target_node, distance in shortest_paths.items():
    print(f"Node {source_node} to Node {target_node}: Distance = {distance:.2f}")

# Save Graph Statistics to a CSV File
graph_data = {
    'node': list(graph.nodes),
    'degree': [deg for _, deg in graph.degree()],
    'clustering_coefficient': [nx.clustering(graph, node) for node in graph.nodes],
}

graph_df = pd.DataFrame(graph_data)
graph_df.to_csv('../results/graph_statistics.csv', index=False)
print("\nGraph statistics saved to '../results/graph_statistics.csv'")

# Clustering Coefficient Distribution
plt.figure(figsize=(10, 6))
plt.hist(graph_df['clustering_coefficient'], bins=np.linspace(0, 1, 20), edgecolor='black')
plt.title("Clustering Coefficient Distribution")
plt.xlabel("Clustering Coefficient")
plt.ylabel("Frequency")
plt.grid(axis='y', linestyle='--')
plt.show()

# Check for Connected Components
connected_components = list(nx.connected_components(graph.to_undirected()))
print(f"\nNumber of connected components: {len(connected_components)}")
if len(connected_components) > 1:
    print("Graph is not fully connected. Consider increasing edge density for better connectivity.")
else:
    print("Graph is fully connected.")

# Path Length Distribution
path_lengths = []
for source in graph.nodes:
    lengths = nx.single_source_dijkstra_path_length(graph, source)
    path_lengths.extend(lengths.values())

plt.figure(figsize=(10, 6))
plt.hist(path_lengths, bins=20, edgecolor='black')
plt.title("Path Length Distribution")
plt.xlabel("Path Length")
plt.ylabel("Frequency")
plt.grid(axis='y', linestyle='--')
plt.show()

# Train Graph Neural Network (GNN) Model
input_dim = 1  # Assuming each node has a single feature
output_dim = 1
model = build_gnn_model(input_dim, output_dim)

# Prepare Data for GNN Training
data_list = []
for node in graph.nodes:
    neighbors = list(graph.neighbors(node))
    edge_index = torch.tensor([[node, neighbor] for neighbor in neighbors], dtype=torch.long).t().contiguous()
    x = torch.ones((len(graph.nodes), 1))  # Node features
    data = torch_geometric.data.Data(x=x, edge_index=edge_index)
    data_list.append(data)

data_loader = DataLoader(data_list, batch_size=16, shuffle=True)

# Train the GNN Model
train_gnn_model(model, data_loader, epochs=10)

# Benchmark ML and Traditional Approaches
from src.evaluation import benchmark_performance
from src.network_simulation import simulate_network_event

# Define models to benchmark (Traditional and ML-based)
models = [run_dijkstra, model]
event_sequence = ['node_failure', 'link_failure', 'link_recovery'] * 5
benchmark_results = benchmark_performance(models, graph, event_sequence)
print("\nBenchmark results saved to '../results/comparison_results.csv'")


# Summary and Insights
Summary and Insights:
- The generated graph has {num_nodes} nodes and approximately {edge_density * num_nodes * (num_nodes - 1) / 2:.0f} edges.
- The degree distribution indicates the presence of both high-degree and low-degree nodes, suggesting some variance in connectivity.
- The clustering coefficient histogram shows how tightly knit the neighborhoods are within the graph.
- The shortest path calculations using Dijkstra's algorithm reveal the efficiency of traditional methods for simple graph structures.
- The graph is {'fully connected' if len(connected_components) == 1 else 'not fully connected'}, which may affect certain models' performance.

Next steps:
- Proceed with model training using ML-based approaches, such as Graph Neural Networks (GNNs) to predict shortest paths or Reinforcement Learning for dynamic pathfinding.
- Perform additional experiments on different graph sizes and edge densities to evaluate the robustness and scalability of both traditional and ML-based approaches.
- Benchmark the performance (accuracy, speed, and scalability) of ISPF/SPF algorithms against ML models by running simulations of network events (e.g., node failures, link failures, and recoveries).
- Develop visualizations and statistical analyses comparing the pathfinding efficiency of each model under different network conditions, and document insights to aid in understanding the strengths and weaknesses of each approach.
- Optimize the hyperparameters of ML models to improve their predictive accuracy and computational efficiency.